In [26]:
import pandas as pd 
import sqlite3

In [27]:
def clean_and_transform(df):

    df["orderdatum"] = pd.to_datetime(df["orderdatum"], errors="coerce")
    df["leveransdatum"] = pd.to_datetime(df["leveransdatum"], errors="coerce")

    df["antal"] = pd.to_numeric(df["antal"], errors="coerce")
    df["pris_per_enhet"] = pd.to_numeric(df["pris_per_enhet"], errors="coerce")

    df = df.dropna(subset=[
        "orderdatum",
        "leveransdatum",
        "antal",
        "pris_per_enhet"
    ])

    df["region"] = df["region"].fillna("Unknown")
    df["betalmetod"] = df["betalmetod"].fillna("Unknown")
    df["leveransstatus"] = df["leveransstatus"].fillna("Unknown")

    df["revenue"] = df["antal"] * df["pris_per_enhet"]
    df["lead_time_days"] = (df["leveransdatum"] - df["orderdatum"]).dt.days

    return df


In [28]:
df = pd.read_csv("../data/raw/nordtech_data.csv")

df = clean_and_transform(df)


In [29]:
df = pd.read_csv("../data/raw/nordtech_data.csv")
df.head()


,order_id,orderrad_id,orderdatum,leveransdatum,produkt_sku,produktnamn,kategori,antal,pris_per_enhet,region,kundtyp,betalmetod,kund_id,leveransstatus,recension_text,recensionsdatum,betyg
0,ORD-2024-00001,ORD-2024-00001-1,2024-05-19,2024-05-22,SKU-WC001,Webbkamera HD,Tillbehör,1,SEK 799,Uppsala,Privat,Kort,KND-53648,Levererad,NaN,NaN,NaN
1,ORD-2024-00002,ORD-2024-00002-1,2024-12-02,5 december 2024,SKU-HB001,USB-C Hub 7-port,Tillbehör,1,549.00,Göteborg,Privat,Swish,KND-84095,Levererad,NaN,NaN,NaN
2,ORD-2024-00003,ORD-2024-00003-1,2024-12-31,2025-01-03,SKU-SD001,Extern SSD 1TB,Lagring,1,1199.00,NaN,Företag,Faktura,KND-91748,Levererad,Stämmer inte överens med produktbeskrivningen.,2025-01-12,2.0
3,ORD-2024-00003,ORD-2024-00003-2,2024-12-31,2025-01-03,SKU-SD002,Extern SSD 500GB,Lagring,10,699 kr,Stockholm,Företag,FAKTURA,KND-91748,Mottagen,"Leveransen tog lite längre än utlovat, men pro...",2025-01-14,3.0
4,ORD-2024-00003,ORD-2024-00003-3,2024-12-31,2025-01-03,SKU-MS001,Trådlös Mus X1,Tillbehör,1,399.00,Stockholm,Företag,Faktura,KND-91748,NaN,NaN,NaN,NaN


In [30]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 2767 entries, 0 to 2766
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         2767 non-null   str    
 1   orderrad_id      2767 non-null   str    
 2   orderdatum       2767 non-null   str    
 3   leveransdatum    2767 non-null   str    
 4   produkt_sku      2767 non-null   str    
 5   produktnamn      2767 non-null   str    
 6   kategori         2767 non-null   str    
 7   antal            2767 non-null   str    
 8   pris_per_enhet   2767 non-null   str    
 9   region           2612 non-null   str    
 10  kundtyp          2767 non-null   str    
 11  betalmetod       2651 non-null   str    
 12  kund_id          2767 non-null   str    
 13  leveransstatus   2673 non-null   str    
 14  recension_text   1355 non-null   str    
 15  recensionsdatum  1355 non-null   str    
 16  betyg            1355 non-null   float64
dtypes: float64(1), str(16)
me

### Convert dates

We convert `orderdatum` and `leveransdatum` to datetime so we can:
- calculate lead time / delivery delay
- extract weekday / month features

`errors="coerce"` converts invalid date strings to `NaT` (missing datetime).


In [31]:
date_cols = ["orderdatum", "leveransdatum"]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")


In [32]:
df.dtypes

order_id                      str
orderrad_id                   str
orderdatum         datetime64[us]
leveransdatum      datetime64[us]
produkt_sku                   str
produktnamn                   str
kategori                      str
antal                         str
pris_per_enhet                str
region                        str
kundtyp                       str
betalmetod                    str
kund_id                       str
leveransstatus                str
recension_text                str
recensionsdatum               str
betyg                     float64
dtype: object

### Convert numeric columns

We ensure numeric columns are stored as numbers to enable calculations (e.g., revenue).
`errors="coerce"` converts invalid numeric strings to `NaN`.


In [33]:
df["antal"] = pd.to_numeric(df["antal"], errors="coerce")
df["pris_per_enhet"] = pd.to_numeric(df["pris_per_enhet"], errors="coerce")


In [34]:
df.dtypes

order_id                      str
orderrad_id                   str
orderdatum         datetime64[us]
leveransdatum      datetime64[us]
produkt_sku                   str
produktnamn                   str
kategori                      str
antal                     float64
pris_per_enhet            float64
region                        str
kundtyp                       str
betalmetod                    str
kund_id                       str
leveransstatus                str
recension_text                str
recensionsdatum               str
betyg                     float64
dtype: object

### Missing values

We inspect missing values per column to decide whether to:
- drop rows (if critical fields are missing)
- fill values (if it makes sense)
- keep as missing (e.g., optional review data)


In [35]:
df.isna().sum()


order_id              0
orderrad_id           0
orderdatum          187
leveransdatum       192
produkt_sku           0
produktnamn           0
kategori              0
antal               150
pris_per_enhet      178
region              155
kundtyp               0
betalmetod          116
kund_id               0
leveransstatus       94
recension_text     1412
recensionsdatum    1412
betyg              1412
dtype: int64

### Handling missing values

We classify columns into:

Critical fields:
- orderdatum
- leveransdatum
- antal
- pris_per_enhet

Rows missing these fields will be removed.

Optional fields:
- region
- betalmetod
- leveransstatus

These will be filled with "Unknown".

Review fields:
- recension_text
- recensionsdatum
- betyg

These are allowed to remain missing.


In [36]:
df = df.dropna(subset=[
    "orderdatum",
    "leveransdatum",
    "antal",
    "pris_per_enhet"
])


In [37]:
df.isna().sum()


order_id              0
orderrad_id           0
orderdatum            0
leveransdatum         0
produkt_sku           0
produktnamn           0
kategori              0
antal                 0
pris_per_enhet        0
region              119
kundtyp               0
betalmetod           94
kund_id               0
leveransstatus       73
recension_text     1076
recensionsdatum    1076
betyg              1076
dtype: int64

In [38]:
df["region"] = df["region"].fillna("Unknown")
df["betalmetod"] = df["betalmetod"].fillna("Unknown")
df["leveransstatus"] = df["leveransstatus"].fillna("Unknown")


In [39]:
df[["region", "betalmetod", "leveransstatus"]].isna().sum()


region            0
betalmetod        0
leveransstatus    0
dtype: int64

Review-related columns are allowed to remain missing because
customer feedback is optional and not required for operational analytics.


In [40]:
df.isna().sum()


order_id              0
orderrad_id           0
orderdatum            0
leveransdatum         0
produkt_sku           0
produktnamn           0
kategori              0
antal                 0
pris_per_enhet        0
region                0
kundtyp               0
betalmetod            0
kund_id               0
leveransstatus        0
recension_text     1076
recensionsdatum    1076
betyg              1076
dtype: int64

### Feature Engineering – Delivery lead time

We calculate delivery lead time in days as the difference between
delivery date and order date.


In [41]:
df["lead_time_days"] = (
    df["leveransdatum"] - df["orderdatum"]
).dt.days


### Feature Engineering – Revenue

Revenue per order row is calculated as quantity multiplied by price per unit.


In [42]:
df["revenue"] = df["antal"] * df["pris_per_enhet"]


### Feature Engineering – Time based features

We extract weekday and month to support time-based sales analysis.


In [43]:
df["order_weekday"] = df["orderdatum"].dt.day_name()
df["order_month"] = df["orderdatum"].dt.month


In [44]:
weekly_sales = (
    df.groupby(pd.Grouper(key="orderdatum", freq="W"))["revenue"]
    .sum()
    .reset_index()
)


In [45]:
df.head()


,order_id,orderrad_id,orderdatum,leveransdatum,produkt_sku,produktnamn,kategori,antal,pris_per_enhet,region,...,betalmetod,kund_id,leveransstatus,recension_text,recensionsdatum,betyg,lead_time_days,revenue,order_weekday,order_month
2,ORD-2024-00003,ORD-2024-00003-1,2024-12-31,2025-01-03,SKU-SD001,Extern SSD 1TB,Lagring,1.0,1199.0,Unknown,...,Faktura,KND-91748,Levererad,Stämmer inte överens med produktbeskrivningen.,2025-01-12,2.0,3,1199.0,Tuesday,12
4,ORD-2024-00003,ORD-2024-00003-3,2024-12-31,2025-01-03,SKU-MS001,Trådlös Mus X1,Tillbehör,1.0,399.0,Stockholm,...,Faktura,KND-91748,Unknown,NaN,NaN,NaN,3,399.0,Tuesday,12
6,ORD-2024-00005,ORD-2024-00005-1,2024-07-01,2024-07-05,SKU-HB001,USB-C Hub 7-port,Tillbehör,10.0,549.0,örebro,...,Faktura,KND-32599,Levererad,NaN,NaN,NaN,4,5490.0,Monday,7
7,ORD-2024-00005,ORD-2024-00005-2,2024-07-01,2024-07-05,SKU-HS001,Headset Pro ANC,Ljud,3.0,1899.0,Örebro,...,FAKTURA,KND-32599,Levererad,Medelmåttig upplevelse.,2024-07-12,3.0,4,5697.0,Monday,7
8,ORD-2024-00005,ORD-2024-00005-3,2024-07-01,2024-07-05,SKU-KB002,Kompakt Tangentbord Mini,Tillbehör,2.0,599.0,Örebro,...,Faktura,KND-32599,Levererad,NaN,NaN,NaN,4,1198.0,Monday,7


### Load cleaned data to SQLite database

We store the transformed dataset in a local SQLite database.

This allows:
- structured storage
- SQL querying
- downstream analytics


In [46]:
import sqlite3

conn = sqlite3.connect("../database/nordtech.db")

df.to_sql(
    "orders_cleaned",
    conn,
    if_exists="replace",
    index=False
)

conn.close()


In [47]:
conn = sqlite3.connect("../database/nordtech.db")

pd.read_sql("SELECT COUNT(*) FROM orders_cleaned", conn)


,COUNT(*)
0,2121


In [48]:
print("Rows in dataframe:", len(df))


Rows in dataframe: 2121


The row count in SQLite matches the transformed dataframe, confirming the load step completed successfully.
